In [ ]:
!mkdir backend


In [ ]:
!touch backend/app.py
!touch backend/utils.py


In [ ]:
!pip install flask joblib numpy pandas


In [ ]:
from google.colab import files
files.upload()


Saving random_forest.pkl to random_forest.pkl


{'random_forest.pkl': b"\x80\x04\x95J\x03\x00\x00\x00\x00\x00\x00\x8c\x10sklearn.pipeline\x94\x8c\x08Pipeline\x94\x93\x94)\x81\x94}\x94(\x8c\x05steps\x94]\x94(\x8c\x06scaler\x94\x8c\x1bsklearn.preprocessing._data\x94\x8c\x0eStandardScaler\x94\x93\x94)\x81\x94}\x94(\x8c\twith_mean\x94\x88\x8c\x08with_std\x94\x88\x8c\x04copy\x94\x88\x8c\x10_sklearn_version\x94\x8c\x051.6.1\x94ub\x86\x94\x8c\x05model\x94\x8c\x18sklearn.ensemble._forest\x94\x8c\x16RandomForestClassifier\x94\x93\x94)\x81\x94}\x94(\x8c\testimator\x94\x8c\x15sklearn.tree._classes\x94\x8c\x16DecisionTreeClassifier\x94\x93\x94)\x81\x94}\x94(\x8c\tcriterion\x94\x8c\x04gini\x94\x8c\x08splitter\x94\x8c\x04best\x94\x8c\tmax_depth\x94N\x8c\x11min_samples_split\x94K\x02\x8c\x10min_samples_leaf\x94K\x01\x8c\x18min_weight_fraction_leaf\x94G\x00\x00\x00\x00\x00\x00\x00\x00\x8c\x0cmax_features\x94N\x8c\x0emax_leaf_nodes\x94N\x8c\x0crandom_state\x94N\x8c\x15min_impurity_decrease\x94G\x00\x00\x00\x00\x00\x00\x00\x00\x8c\x0cclass_weight\x94

In [ ]:
!mkdir models
!mv random_forest.pkl models/


mkdir: cannot create directory ‘models’: File exists
mv: cannot stat 'random_forest.pkl': No such file or directory


In [ ]:
!find /content -name "random_forest.pkl"


/content/models/random_forest.pkl


In [ ]:
!ls models


random_forest.pkl


In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np
import os   # ✅ THIS LINE WAS MISSING


In [ ]:
%%writefile backend/app.py
from flask import Flask, request, jsonify
import joblib
import numpy as np
import os

Overwriting backend/app.py


In [ ]:
# -----------------------------
# Initialize Flask application
# -----------------------------
app = Flask(__name__)


In [ ]:
# -----------------------------
# Load trained ML model
# -----------------------------
MODEL_PATH = "models/random_forest.pkl"

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError("Model file not found. Please check models/random_forest.pkl")

model = joblib.load(MODEL_PATH)

In [ ]:
# -----------------------------
# Health check route
# -----------------------------
@app.route("/", methods=["GET"])
def home():
    return jsonify({
        "status": "Backend is running",
        "message": "ExoHabitAI Flask API"
    })

In [ ]:
# -----------------------------
# Prediction endpoint
# -----------------------------
@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()

    if data is None:
        return jsonify({
            "status": "error",
            "message": "No JSON data provided"
        }), 400

    try:
        features = [
            data["pl_rade"],
            data["pl_bmasse"],
            data["pl_orbsmax"],
            data["pl_eqt"],
            data["pl_dens"],
            data["st_teff"],
            data["st_lum"],
            data["habitability_index"],
            data["stellar_compatibility"],
            data["orbital_stability"]
        ]

        features_array = np.array(features).reshape(1, -1)

        prediction = model.predict(features_array)[0]
        probability = model.predict_proba(features_array)[0][1]

        result = "Potentially Habitable" if prediction == 1 else "Not Habitable"

        return jsonify({
            "status": "success",
            "prediction": result,
            "confidence_score": round(float(probability), 4)
        })

    except KeyError as e:
        return jsonify({
            "status": "error",
            "message": f"Missing parameter: {str(e)}"
        }), 400

    except Exception as e:
        return jsonify({
            "status": "error",
            "message": str(e)
        }), 500

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
